In [1]:
pip install PyMuPDF

   ---------------------------------------- 0.0/16.5 MB ? eta -:--:--
    --------------------------------------- 0.3/16.5 MB ? eta -:--:--
   - -------------------------------------- 0.8/16.5 MB 2.4 MB/s eta 0:00:07
   -- ------------------------------------- 1.0/16.5 MB 2.2 MB/s eta 0:00:08
   --- ------------------------------------ 1.6/16.5 MB 2.1 MB/s eta 0:00:08
   ----- ---------------------------------- 2.1/16.5 MB 2.1 MB/s eta 0:00:08
   ----- ---------------------------------- 2.4/16.5 MB 2.0 MB/s eta 0:00:07
   ------ --------------------------------- 2.9/16.5 MB 2.0 MB/s eta 0:00:07
   ------- -------------------------------- 3.1/16.5 MB 2.0 MB/s eta 0:00:07
   -------- ------------------------------- 3.7/16.5 MB 2.0 MB/s eta 0:00:07
   ---------- ----------------------------- 4.2/16.5 MB 2.0 MB/s eta 0:00:07
   ---------- ----------------------------- 4.5/16.5 MB 2.0 MB/s eta 0:00:07
   ------------ --------------------------- 5.0/16.5 MB 2.0 MB/s eta 0:00:06
   ----------

In [13]:
import fitz  
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.tools import Tool
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
from langchain.agents import initialize_agent, Tool, AgentType
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.memory import ConversationBufferMemory
from langchain_community.embeddings import HuggingFaceEmbeddings
import os

In [3]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text() for page in doc])
    return text

text = extract_text_from_pdf("C:\Disk D\Semester 4 2\LLMs\Financial_Agent\The Total Money Makeover - Dave Ramsey.pdf")

In [4]:
text

'\nThe\nTOTAL MONEY\nMAKEOVER\n \n\nOther Books from Dave Ramsey\n \nThe Total Money Makeover Workbook\n \nThe Total Money Makeover Spanish Edition  \n  (La Transformación Total de su Dinero)\n \nFinancial Peace Revisited\n \nFinancial Peace Planner\n \nFoundations in Personal Finance \n  (High-school Curriculum)\n \nTranquilidad Financiera \n  (Spanish edition of Financial Peace )\n \nMore Than Enough\n \nMore Than Enough Planner\n \nLife Lessons with Junior\n(Children’s Series)\n \n\nThe\nTOTAL MONEY\nMAKEOVER\nA Proven Plan for Financial Fitness\n \nDave Ramsey\n \n\n© 2003, 2007, 2009 by Dave Ramsey\n \nAll rights reserved. No portion of this book may be reproduced, stored in a retrieval system, or\ntransmitted in any form or by any means—electronic, mechanical, photocopy, recording,\nscanning, or other—except for brief quotations in critical reviews or articles, without the prior\nwritten permission of the publisher.\n \nPublished in Nashville, Tennessee, by Thomas Nelson. Thomas 

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.create_documents([text])

In [7]:
embeddings_model = HuggingFaceEmbeddings(
    model_name="jinaai/jina-embeddings-v2-base-en",
    model_kwargs={'device': 'cpu'}  # Use 'cuda' if you have GPU
)

C:\Users\harsh\AppData\Local\Temp\ipykernel_40144\159041887.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(
c:\Disk D\Semester 4 2\LLMs\Langchain_crash_course\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-base-en and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'enc

In [10]:
vectorstore = FAISS.from_documents(docs, embeddings_model)
vectorstore.save_local("faiss_index")

In [11]:
vectorstore = FAISS.load_local("faiss_index", embeddings_model, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever()

In [14]:
llm = ChatGroq(model="llama3-8b-8192")

In [15]:
retrieval_qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

In [16]:
def self_evaluate(input_text):
    parts = input_text.split("|||")
    query = parts[0]
    response = parts[1]
    sources = parts[2] if len(parts) > 2 else ""
    
    evaluation_prompt = f"""
    Evaluate the following response to the query:
    
    QUERY: {query}
    RESPONSE: {response}
    SOURCES: {sources}
    
    Assess based on:
    1. Factual accuracy (Does it match the sources?)
    2. Completeness (Does it address all aspects of the query?)
    3. Relevance (Is the information relevant to the query?)
    4. Hallucination (Does it contain information not supported by sources?)
    
    Return a confidence score from 0-10 and explanation.
    """
    
    evaluation = llm.predict(evaluation_prompt)
    return evaluation

In [17]:
tools = [
    Tool(
        name="Article Retrieval",
        func=lambda q: retrieval_qa_chain({"query": q})["result"],
        description="Retrieve knowledge from the article database."
    ),
]

In [18]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

C:\Users\harsh\AppData\Local\Temp\ipykernel_40144\3353824168.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [19]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    memory=memory
)

C:\Users\harsh\AppData\Local\Temp\ipykernel_40144\2061683320.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [20]:
def get_evaluated_response(query):
    response = agent.run(query)
    
    try:
        result = retrieval_qa_chain({"query": query})
        sources = [doc.page_content for doc in result.get("source_documents", [])]
        sources_text = "\n".join(sources)
    except Exception as e:
        sources_text = "No sources available"
    
    evaluation = self_evaluate(f"{query}|||{response}|||{sources_text}")
    
    return {
        "query": query,
        "response": response,
        "evaluation": evaluation,
        "sources": sources_text
    }

In [21]:
def transparent_response(query):
    result = get_evaluated_response(query)
    
    return f"""
    Response: {result['response']}
    
    Confidence assessment: {result['evaluation']}
    """

In [24]:
print(transparent_response("What are the myths about debt?"))



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Article Retrieval",
  "action_input": "myths about debt"
}
```

Observation: According to the text, some debt myths are:

1. Debt is how you become wealthy.
Thought:Action:
```
{
  "action": "Article Retrieval",
  "action_input": "common debt myths"
}
```


Observation: According to the text, the author mentions the following debt myths:

1. Debt is a tool: The author argues that debt is not a tool, but rather a sign of decline and failure.
2. Borrowing lots of money and not saving any is a good strategy: The author suggests that this approach is a result of hubris and a false sense of invincibility, and is not a sustainable or responsible financial strategy.

These are the two debt myths that are explicitly mentioned in the text.
Thought:Action:
```
{
  "action": "Article Retrieval",
  "action_input": "most common debt myths"
}
```


Observation: According to the context, in Chapter 3 of the book "The Total Money Makeo

In [1]:
# %% Install dependencies
#pip install PyMuPDF langchain langchain_groq langchain_community faiss-cpu

# %% Import required libraries
import fitz  
import re
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent, AgentType, Tool

# %% Load LLM model
llm = ChatGroq(model="llama3-8b-8192")

# %% Function to extract structured text from PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    current_chapter = None

    for page in doc:
        page_text = page.get_text()
        
        # Detecting chapters using regex (adjust as per formatting)
        chapter_match = re.search(r'CHAPTER\s+\d+', page_text, re.IGNORECASE)
        if chapter_match:
            current_chapter = chapter_match.group(0)
        
        # Tagging text with chapter information
        if current_chapter:
            text += f"\n\n## {current_chapter}\n\n" + page_text

    return text

# %% Extract and preprocess text
pdf_path = "C:/Disk D/Semester 4 2/LLMs/Financial_Agent/The Total Money Makeover - Dave Ramsey.pdf"
text = extract_text_from_pdf(pdf_path)

# %% Chunk the extracted text for vector storage
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100, separators=["\n\n##", "\n\n"]
)
docs = text_splitter.create_documents([text])

# %% Load embedding model
embeddings_model = HuggingFaceEmbeddings(
    model_name="jinaai/jina-embeddings-v2-base-en",
    model_kwargs={'device': 'cpu'}  # Use 'cuda' if you have GPU
)

# %% Store chunks in FAISS vector database
vectorstore = FAISS.from_documents(docs, embeddings_model)
vectorstore.save_local("faiss_index")

# %% Load stored FAISS vector database
vectorstore = FAISS.load_local("faiss_index", embeddings_model, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever()

# %% Function to retrieve relevant sections based on chapter and topic
def retrieve_chapter_topic(chapter, topic):
    query = f"Find information in {chapter} about {topic}."
    results = retriever.get_relevant_documents(query)
    
    # If results are insufficient, expand the search to the full document
    if len(results) < 3:
        results += retriever.get_relevant_documents(topic)
    
    return results

# %% Function to generate a structured lesson-style response
def teach_topic(chapter, topic):
    docs = retrieve_chapter_topic(chapter, topic)
    
    source_text = "\n".join([doc.page_content for doc in docs])

    teaching_prompt = f"""
    You are an expert finance professor. Teach the topic "{topic}" from "{chapter}" in a structured, engaging manner.

    Format the response as:
    1. **Introduction** - Explain why this topic is important.
    2. **Key Concepts** - Define and explain the core ideas.
    3. **Real-World Examples** - Provide case studies or applications.
    4. **Common Misconceptions** - Address common misunderstandings.
    5. **Conclusion** - Summarize the key takeaways.

    Use the following reference material:
    {source_text}
    """

    response = llm.predict(teaching_prompt)
    return response

# %% Query function to get structured teaching response
def get_teaching_response(chapter, topic):
    response = teach_topic(chapter, topic)
    
    return f"""
    **Topic:** {topic}  
    **Chapter:** {chapter}  
    
    {response}
    """

# %% Example Query
print(get_teaching_response("CHAPTER 4: Sources of Value Creation", "Risk and Return"))


C:\Users\harsh\AppData\Local\Temp\ipykernel_11708\2287425488.py:49: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(
c:\Disk D\Semester 4 2\LLMs\Langchain_crash_course\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-base-en and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'e


    **Topic:** Risk and Return  
    **Chapter:** CHAPTER 4: Sources of Value Creation  
    
    **Introduction**
Risk and Return is a fundamental concept in finance that helps investors understand the trade-off between the potential gains and potential losses of an investment. As we explore this topic, we'll delve into the world of risk and return, examining the relationships between investment returns, risk, and time. Understanding these relationships is crucial for making informed investment decisions and creating a successful investment strategy.

**Key Concepts**

* **Risk**: The possibility of losing money or incurring a loss due to market fluctuations, economic downturns, or other factors.
* **Return**: The profit or gain generated by an investment, typically measured as a percentage or as a multiple of the initial investment.
* **Risk-Free Rate**: The rate of return on a risk-free investment, such as a U.S. Treasury bond, that reflects the market's expectations of future inte

In [2]:
Markdown(print(get_teaching_response("CHAPTER 4: Sources of Value Creation", "Risk and Return")))

NameError: name 'Markdown' is not defined

In [4]:
from IPython.display import Markdown
Markdown(print(get_teaching_response("CHAPTER 4: Sources of Value Creation", "Risk and Return")))



    **Topic:** Risk and Return  
    **Chapter:** CHAPTER 4: Sources of Value Creation  
    
    **Introduction**

Understanding the relationship between risk and return is crucial for investors seeking to maximize their financial returns while minimizing potential losses. In this chapter, we will explore the concept of risk and return, and how it applies to various investment opportunities. We will also examine common misconceptions and real-world examples to illustrate the importance of considering risk and return when making investment decisions.

**Key Concepts**

* **Risk**: The possibility of loss or damage due to uncertainty or unpredictability.
* **Return**: The profit or gain generated by an investment.
* **Risk-Return Tradeoff**: The relationship between the level of risk taken and the potential return earned.
* **Expected Return**: The average return expected from an investment over a given period.
* **Standard Deviation**: A measure of the volatility or uncertainty of an 

<IPython.core.display.Markdown object>

In [5]:
# %% Install dependencies
# pip install PyMuPDF langchain langchain_groq langchain_community faiss-cpu

# %% Import required libraries
import fitz  
import re
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent, AgentType, Tool

# %% Load LLM model
llm = ChatGroq(model="llama3-8b-8192")

# %% Function to extract structured text from PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    current_chapter = None

    for page in doc:
        page_text = page.get_text()
        
        # Detecting chapters using regex (adjust as per formatting)
        chapter_match = re.search(r'CHAPTER\s+\d+', page_text, re.IGNORECASE)
        if chapter_match:
            current_chapter = chapter_match.group(0)
        
        # Tagging text with chapter information
        if current_chapter:
            text += f"\n\n## {current_chapter}\n\n" + page_text

    return text

# %% Extract and preprocess text
pdf_path = "C:/Disk D/Semester 4 2/LLMs/Financial_Agent/The Total Money Makeover - Dave Ramsey.pdf"
text = extract_text_from_pdf(pdf_path)

# %% Chunk the extracted text for vector storage
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100, separators=["\n\n##", "\n\n"]
)
docs = text_splitter.create_documents([text])

# %% Load embedding model
embeddings_model = HuggingFaceEmbeddings(
    model_name="jinaai/jina-embeddings-v2-base-en",
    model_kwargs={'device': 'cpu'}  # Use 'cuda' if you have GPU
)

# %% Store chunks in FAISS vector database
#vectorstore = FAISS.from_documents(docs, embeddings_model)
#vectorstore.save_local("faiss_index")

# %% Load stored FAISS vector database
vectorstore = FAISS.load_local("faiss_index", embeddings_model, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever()

# %% Function to retrieve relevant sections based on chapter and topic
def retrieve_chapter_topic(chapter, topic):
    query = f"Find information in {chapter} about {topic}."
    results = retriever.get_relevant_documents(query)
    
    # If results are insufficient, expand the search to the full document
    if len(results) < 3:
        results += retriever.get_relevant_documents(topic)
    
    return results

# %% Function to generate a structured lesson-style response in Markdown format
def teach_topic(chapter, topic):
    docs = retrieve_chapter_topic(chapter, topic)
    
    source_text = "\n".join([doc.page_content for doc in docs])

    teaching_prompt = f"""
    You are an expert finance professor. Teach the topic "{topic}" from "{chapter}" in a structured, engaging manner.

    Format the response in Markdown as:
    # 📘 Topic: {topic}
    ## 📖 Chapter: {chapter}

    ### 🔹 **Introduction**
    Explain why this topic is important.

    ### 🔹 **Key Concepts**
    1. Define and explain the core ideas.
    2. Highlight important sub-concepts.

    ### 🔹 **Real-World Examples**
    - Provide case studies or applications.
    
    ### 🔹 **Common Misconceptions**
    ❌ Address misunderstandings or incorrect beliefs.
    
    ### 🔹 **Conclusion**
    Summarize the key takeaways.

    Use the following reference material:
    {source_text}
    """

    response = llm.predict(teaching_prompt)
    
    # Wrapping response in Markdown format
    markdown_response = f"```markdown\n{response}\n```"
    return markdown_response

# %% Query function to get structured Markdown output
def get_teaching_response(chapter, topic):
    response = teach_topic(chapter, topic)
    
    return response  # Already formatted in Markdown

# %% Example Query
print(get_teaching_response("CHAPTER 4: Sources of Value Creation", "Risk and Return"))


Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-base-en and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.10.intermediate.dense.bias', 'encoder.layer.10.intermediate.dense.weight', 'encoder.layer.10.output.LayerNorm.bias', 'encoder.layer.10.output.LayerNorm.weight', 'encoder.layer.10.output.dense.bias', 'encoder.layer.10.output.dense.weight', 'encoder.layer.11.intermediate.dense.bias', 'encoder.layer.11.intermedi

```markdown
# 📘 Topic: Risk and Return
## 📖 Chapter: CHAPTER 4: Sources of Value Creation

### 🔹 **Introduction**
Risk and return are two fundamental concepts in finance that are closely intertwined. Understanding the relationship between risk and return is crucial for investors, as it helps them make informed decisions about their investments. In this chapter, we will explore the key concepts, real-world examples, and common misconceptions related to risk and return.

### 🔹 **Key Concepts**
1. **Risk**: Risk refers to the uncertainty or volatility of an investment's returns. It can be measured in terms of standard deviation, beta, or expected shortfall.
2. **Return**: Return refers to the profit or gain generated by an investment. It can be measured in terms of expected return, actual return, or return on investment (ROI).
3. **Risk-Return Tradeoff**: The risk-return tradeoff refers to the idea that investments with higher returns typically come with higher levels of risk, and vice ve

In [6]:
Markdown(get_teaching_response("CHAPTER 4: Sources of Value Creation", "Risk and Return"))

```markdown
# 📘 Topic: Risk and Return
## 📖 Chapter: CHAPTER 4: Sources of Value Creation

### 🔹 **Introduction**
Risk and return are fundamental concepts in finance that help us understand the trade-off between the potential benefits of an investment and the potential losses. In this topic, we'll explore the relationship between risk and return, and how it affects our investment decisions.

### 🔹 **Key Concepts**

1. **Risk**: The possibility of loss or injury. In finance, risk refers to the uncertainty of an investment's future value or returns.
2. **Return**: The income or profit generated by an investment. In finance, return refers to the total return on an investment, including interest, dividends, and capital appreciation.
3. **Expected Return**: The return that an investor expects to earn from an investment, based on its historical performance and market conditions.
4. **Risk-Return Tradeoff**: The idea that investments with higher potential returns typically come with higher levels of risk, and vice versa.

### 🔹 **Real-World Examples**

* The case study of Doug and his wife, who paid off their debt and achieved financial peace by following the Baby Steps and establishing a budget.
* The story of Brian and Tammy McKinley, who became debt-free and achieved financial stability by following the Total Money Makeover.
* The example of Sara and Bob, who paid cash for a family vacation and achieved financial freedom by living below their means.

### 🔹 **Common Misconceptions**

* ❌ The myth that debt is how you become wealthy. In reality, debt can be a significant source of financial stress and should be avoided whenever possible.
* ❌ The myth that money myths, such as "I don't have a problem" or "debt is how you become wealthy," are true. In reality, these myths can lead to financial ruin if not addressed.

### 🔹 **Conclusion**
In conclusion, risk and return are fundamental concepts in finance that help us understand the trade-off between the potential benefits of an investment and the potential losses. By understanding the risk-return relationship, investors can make more informed decisions about their investments and achieve their financial goals. Remember, risk and return are not mutually exclusive, and a balanced approach to investing can help investors achieve financial stability and peace of mind.
```

In [7]:
# %% Function to generate a structured lesson-style response with a quiz in Markdown format
def teach_topic_with_quiz(chapter, topic):
    docs = retrieve_chapter_topic(chapter, topic)
    
    source_text = "\n".join([doc.page_content for doc in docs])

    teaching_prompt = f"""
    You are an expert finance professor. Teach the topic "{topic}" from "{chapter}" in a structured, engaging manner.

    **Strictly use finance and investment-related examples, avoiding personal finance cases.**

    Format the response in Markdown as:
    ```markdown
    # 📘 Topic: {topic}
    ## 📖 Chapter: {chapter}

    ### 🔹 **Introduction**
    Explain why this topic is important in financial decision-making.

    ### 🔹 **Key Concepts**
    1. Define and explain the core ideas.
    2. Highlight important sub-concepts related to finance and investing.

    ### 🔹 **Real-World Examples**
    - Provide case studies on **stock markets, corporate finance, risk management**.
    - Avoid personal finance examples like budgeting or debt repayment.

    ### 🔹 **Common Misconceptions**
    ❌ Address misunderstandings in finance and investing.

    ### 🔹 **Conclusion**
    Summarize key takeaways about risk-return tradeoff and decision-making.

    ---
    💡 *Key Takeaway:* A well-balanced approach to risk and return ensures financial stability.

    ---
    ## 📝 Quiz Time!
    - **Question 1:** What is the primary difference between systematic and unsystematic risk?  
    - **Question 2:** Which of the following investments has the highest risk: (A) Government Bonds, (B) Blue-chip Stocks, (C) Startups?  
    - **Question 3:** True or False: Diversification can completely eliminate all types of investment risk.  
    - **Question 4:** What is the risk-return tradeoff, and why is it important for investors?  

    ---
    ### ✅ **Quiz Answers**
    1. Systematic risk affects the entire market (e.g., economic downturns), while unsystematic risk is specific to a company or industry.  
    2. (C) Startups have the highest risk due to their uncertainty and volatility.  
    3. False – Diversification reduces risk but cannot eliminate market-wide risks.  
    4. The risk-return tradeoff is the balance between potential rewards and risks in investments; higher returns often come with higher risk.  

    ---
    📚 *Test your understanding and revise key concepts before moving ahead!*
    ```

    Use the following reference material:
    {source_text}
    """

    response = llm.predict(teaching_prompt)
    
    # Ensure Markdown format
    markdown_response = f"```markdown\n{response}\n```"
    return markdown_response

# %% Query function to get structured Markdown output with a quiz
def get_teaching_response_with_quiz(chapter, topic):
    response = teach_topic_with_quiz(chapter, topic)
    
    return response  # Already formatted in Markdown

# %% Example Query


In [8]:
Markdown(get_teaching_response_with_quiz("CHAPTER 4: Sources of Value Creation", "Risk and Return"))

```markdown
📘 Topic: Risk and Return
## 📖 Chapter: CHAPTER 4: Sources of Value Creation

### 🔹 **Introduction**
Risk and return are two fundamental concepts in finance that are intricately linked. Understanding the relationship between risk and return is crucial for investors, as it helps them make informed decisions about their investments. In this chapter, we will explore the key concepts, real-world examples, and common misconceptions related to risk and return.

### 🔹 **Key Concepts**
1. **Risk**: The uncertainty or unpredictability of an investment's outcome. Risk can be classified into two types: systematic risk (affecting the entire market) and unsystematic risk (specific to a company or industry).
2. **Return**: The profit or gain generated by an investment, typically expressed as a percentage.
3. **Risk-Return Tradeoff**: The balance between potential returns and risks in investments. Higher returns often come with higher risks.

### 🔹 **Real-World Examples**

* **Stock Markets**: Investors in the stock market face both systematic and unsystematic risks. Systematic risks include market-wide events such as recessions or economic downturns, while unsystematic risks are specific to individual companies, such as product recalls or management changes.
* **Corporate Finance**: Companies take on various types of risks, including credit risk (default risk), market risk (interest rate or commodity price risk), and operational risk (risk of loss due to internal or external factors). A well-managed company will balance its risks and returns to achieve its financial goals.
* **Risk Management**: Investors and companies use various techniques to manage risk, such as diversification, hedging, and insurance. These strategies aim to reduce the impact of adverse events and increase the potential for returns.

### 🔹 **Common Misconceptions**
❌ The myth that risk and return are mutually exclusive is a common misconception. In reality, risk and return are interconnected, and investors must balance their risk tolerance with their return expectations.

### 🔹 **Conclusion**
A well-balanced approach to risk and return is essential for investors and companies alike. By understanding the relationship between risk and return, investors can make informed decisions about their investments and achieve their financial goals. The risk-return tradeoff is a crucial concept in finance, and it is essential to consider both the potential returns and risks of an investment before making a decision.

---
💡 *Key Takeaway:* A well-balanced approach to risk and return ensures financial stability.

---
## 📝 Quiz Time!
- **Question 1:** What is the primary difference between systematic and unsystematic risk?  
- **Question 2:** Which of the following investments has the highest risk: (A) Government Bonds, (B) Blue-chip Stocks, (C) Startups?  
- **Question 3:** True or False: Diversification can completely eliminate all types of investment risk.  
- **Question 4:** What is the risk-return tradeoff, and why is it important for investors?  

---
### ✅ **Quiz Answers**
1. Systematic risk affects the entire market (e.g., economic downturns), while unsystematic risk is specific to a company or industry.  
2. (C) Startups have the highest risk due to their uncertainty and volatility.  
3. False – Diversification reduces risk but cannot eliminate market-wide risks.  
4. The risk-return tradeoff is the balance between potential rewards and risks in investments; higher returns often come with higher risk.
```